In [ ]:
import copy

import isambard

# Helix Pair

In [ ]:
test_structure = isambard.ampal.convert_pdb_to_ampal('4uuc.pdb')

In [ ]:
sol1 = test_structure[0]

In [ ]:
sol1.helices.sequences

In [ ]:
rep_hels = test_structure.helices[6:8]

In [ ]:
rep_hels.relabel_all()

In [ ]:
hel_pair = isambard.specifications.HelixPair(aas=(8, 10))

In [ ]:
hel_pair.sequences

In [ ]:
rep_hels.sequences

In [ ]:
hel_pair.relabel_all()

In [ ]:
hel_pair.pack_new_sequences(['PIHEAVKR', 'RECMEILLAN',])

In [ ]:
hel_pair.sequences

In [ ]:
class HelPairOpt(isambard.specifications.HelixPair):
    def __init__(self, rad, zshift, phi1, phi2, sp, op):
        super().__init__(aas=(8, 10), axis_distances=(-rad, rad), z_shifts=(0, zshift),
                         phis=(phi1, phi2), splays=(0, sp), off_plane=(0, op))
        self.relabel_all()

In [ ]:
isambard.external_programs.run_profit(rep_hels.pdb, hel_pair.pdb, path1=False, path2=False)

In [ ]:
opt = isambard.optimisation.DE_RMSD(
    HelPairOpt, rep_hels.pdb)
opt.parameters(hel_pair.sequences,
          [3, 0, 0, 0, 0, 180],
          [3, 6, 180, 180, 45, 90],
          ['var0', 'var1', 'var2', 'var3', 'var4', 'var5'])
opt.run_opt(50, 50, 4)

In [ ]:
best_params = opt.parse_individual(opt.halloffame[0])

In [ ]:
best = HelPairOpt(*best_params)
best.pack_new_sequences(hel_pair.sequences)

# Solenoid

In [ ]:
target = isambard.ampal.Assembly()

In [ ]:
test_structure = isambard.ampal.convert_pdb_to_ampal('4uuc.pdb')

In [ ]:
cha = test_structure[0]

In [ ]:
cha.sequence

In [ ]:
target.append(cha.get_slice_from_res_id('68', '75'))

In [ ]:
target.append(cha.get_slice_from_res_id('78', '87'))

In [ ]:
target.append(cha.get_slice_from_res_id('101', '108'))

In [ ]:
target.append(cha.get_slice_from_res_id('111', '120'))

In [ ]:
target.append(cha.get_slice_from_res_id('134', '141'))

In [ ]:
target.append(cha.get_slice_from_res_id('144', '153'))

In [ ]:
target.append(cha.get_slice_from_res_id('166', '173'))

In [ ]:
target.append(cha.get_slice_from_res_id('176', '185'))

In [ ]:
target.append(cha.get_slice_from_res_id('199', '206'))

In [ ]:
target.append(cha.get_slice_from_res_id('209', '218'))

In [ ]:
target.append(cha.get_slice_from_res_id('231', '238'))

In [ ]:
target.append(cha.get_slice_from_res_id('241', '250'))

In [ ]:
target.sequences

In [ ]:
rep_unit = HelPairOpt(*best_params)

In [ ]:
class SolenoidOpt(isambard.specifications.Solenoid):
    def __init__(self, in_ru, repeats, rad, rise, rot_ang, xr, yr, zr):
        ru = copy.deepcopy(in_ru)
        ru.rotate(xr, (1, 0, 0))
        ru.rotate(yr, (0, 1, 0))
        ru.rotate(zr, (0, 0, 1))
        super().__init__(ru, repeats, rad, rise, rot_ang, 'left')

In [ ]:
target.relabel_polymers()

In [ ]:
opt2 = isambard.optimisation.DE_RMSD(
    SolenoidOpt, target.pdb)
opt2.parameters(target.sequences,
          [35, 10, 0, 0, 0, 0],
          [10, 8.0, 200, 200, 200, 200],
          [rep_unit, 6, 'var0', 'var1', 'var2', 'var3', 'var4', 'var5'])
opt2.run_opt(40, 100, 4)

In [ ]:
best = SolenoidOpt(*opt2.parse_individual(opt2.halloffame[0]))

In [ ]:
best.pack_new_sequences(target.sequences)